In [1]:
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from bs4 import BeautifulSoup, Comment
from fake_useragent import UserAgent
import time
import os
import requests, re
import random
import string
import pickle

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [3]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.basketball-reference.com/boxscores/?month=10&day=25&year=2016")

In [4]:
first_half_box_score_urls = []
second_half_box_score_urls = []

In [5]:
def getting_box_scores(box_score_urls):
    links = driver.find_elements_by_xpath('//*[@id="content"]/div[3]//a[@href]')
    for link in links:
        if 'boxscores' in link.get_attribute('href') and 'Final' in link.text:
            box_score_urls.append(link.get_attribute('href'))

In [ ]:
while (int(driver.find_element_by_xpath('//*[@id="content"]/h1').text[-4:]) <= 2016):
    getting_box_scores(first_half_box_score_urls)
    next_day = driver.find_element_by_xpath('//*[@id="content"]/div[1]/a[2]').get_attribute('href')
    driver.get(next_day)
    time.sleep(5)

In [6]:
getting_box_scores(first_half_box_score_urls)

In [ ]:
driver.get("https://www.basketball-reference.com/boxscores/?month=1&day=1&year=2017")

In [ ]:
while (int(driver.find_element_by_xpath('//*[@id="content"]/h1').text[-4:]) <= 2017 and int(driver.find_element_by_xpath('//*[@id="content"]/h1').text[-8:-6]) <= 12 and int(next_day[-17]) <= 6):
    getting_box_scores(second_half_box_score_urls)
    next_day = driver.find_element_by_xpath('//*[@id="content"]/div[1]/a[2]').get_attribute('href')
    driver.get(next_day)
    time.sleep(5)

In [1]:
half_way_point_url = "https://www.basketball-reference.com/boxscores/?month=1&day=1&year=2017"
half_way_point_url[-17]

'1'

In [ ]:
driver.get(box_score_urls[0])

In [24]:
with open('player_stats_each_game.csv', 'w') as player_stats_per_game_file:

    for box_score_url in first_half_box_score_urls[2:3]:
        driver.get(box_score_url)
        time.sleep(10)
    
        date = box_score_url.split('/')[-1][0:8]
        year = int(date[0:4])
        month = int(date[4:6])
        day = int(date[6:8])
    
        away_team = driver.find_element_by_xpath('//*[@id="line_score"]/tbody/tr[2]/td[1]/a').text
        home_team = driver.find_element_by_xpath('//*[@id="line_score"]/tbody/tr[3]/td[1]/a').text
        teams = [away_team, home_team]

        for team in teams:
            print (team)
            for i in range(1,len(driver.find_elements_by_xpath('//*[@id="box_'+team.lower()+'_basic"]/tbody//tr'))+1):
                players = {}
                if i != 6:
                    player_name = driver.find_element_by_xpath('//*[@id="box_'+team.lower()+'_basic"]/tbody/tr['+str(i)+']/th').text
                    player_stats = driver.find_elements_by_xpath('//*[@id="box_'+team.lower()+'_basic"]/tbody/tr['+str(i)+']//td')
                
                    home_player = player_name+'_'+str(month)+'/'+str(day)+'/'+str(year)+'_vs'+away_team
                    away_player = player_name+'_'+str(month)+'/'+str(day)+'/'+str(year)+'_@'+home_team
                
                    away_points = driver.find_element_by_xpath('//*[@id="box_'+away_team.lower()+'_basic"]/tfoot/tr/td[19]').text
                    home_points = driver.find_element_by_xpath('//*[@id="box_'+home_team.lower()+'_basic"]/tfoot/tr/td[19]').text
                
                    if len(player_stats) > 1:
                        if team == away_team:
                            players[away_player] = [player_name, team, year, month, day, away_team, home_team]
                        elif team == home_team:
                            players[home_player] = [player_name, team, year, month, day, away_team, home_team]
                        
                        for stat in player_stats:
                            if team == away_team:
                                players[away_player].append(stat.text)
                                
                            elif team == home_team:
                                players[home_player].append(stat.text)
                        
                        if team == away_team:
                            if int(away_points) > int(home_points):
                                players[away_player].append(1)
                            else:
                                players[away_player].append(0)
                    
                        if team == home_team:
                            if int(away_points) > int(home_points):
                                players[home_player].append(0)
                            else:
                                players[home_player].append(1)
                
                    else:
                        continue
                    
                    for key, value in players.items():
                        player_stats_per_game_file.write('{0}, {1}\n'.format(key, value))
                    
                else:
                    continue

UTA
POR


In [22]:
players.keys()

dict_keys([])

In [10]:
players_df = pd.DataFrame.from_dict(players)
players_df = players_df.T
players_df

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
Al-Farouq Aminu_10/25/2016_vsUTA,Al-Farouq Aminu,POR,2016,10,25,UTA,POR,28:24,3,6,...,4,4,1,1,0,1,0,9,0,1
Allen Crabbe_10/25/2016_vsUTA,Allen Crabbe,POR,2016,10,25,UTA,POR,29:09,6,10,...,3,3,3,0,0,2,2,18,+14,1
Boris Diaw_10/25/2016_@POR,Boris Diaw,UTA,2016,10,25,UTA,POR,32:10,1,8,...,2,3,0,0,0,1,5,2,+1,0
C.J. McCollum_10/25/2016_vsUTA,C.J. McCollum,POR,2016,10,25,UTA,POR,34:36,8,16,...,1,2,3,1,0,3,3,25,+13,1
Damian Lillard_10/25/2016_vsUTA,Damian Lillard,POR,2016,10,25,UTA,POR,35:25,13,20,...,8,9,6,0,0,4,3,39,+1,1
Dante Exum_10/25/2016_@POR,Dante Exum,UTA,2016,10,25,UTA,POR,8:20,2,4,...,1,2,0,0,0,0,2,5,-18,0
Ed Davis_10/25/2016_vsUTA,Ed Davis,POR,2016,10,25,UTA,POR,24:07,2,3,...,4,7,1,0,1,0,2,6,+21,1
Evan Turner_10/25/2016_vsUTA,Evan Turner,POR,2016,10,25,UTA,POR,26:01,1,7,...,5,5,5,1,1,0,1,3,+9,1
George Hill_10/25/2016_@POR,George Hill,UTA,2016,10,25,UTA,POR,38:45,7,18,...,2,2,6,2,1,1,2,19,+2,0
Jeff Withey_10/25/2016_@POR,Jeff Withey,UTA,2016,10,25,UTA,POR,4:51,0,0,...,1,1,0,1,1,0,1,0,-11,0


In [13]:
type(players_df.iloc[0,8])

str

In [32]:
with open('player_stats_each_game.csv', 'r') as f:
    player_reader = csv.reader(f, delimiter=',')
    for row in player_reader:
        print (row)

['Rudy Gobert_10/25/2016_@POR', " ['Rudy Gobert'", " 'UTA'", ' 2016', ' 10', ' 25', " 'UTA'", " 'POR'", " '40:57'", " '4'", " '7'", " '.571'", " '0'", " '0'", " ''", " '4'", " '4'", " '1.000'", " '4'", " '10'", " '14'", " '2'", " '0'", " '2'", " '3'", " '3'", " '12'", " '+7'", ' 0]']
['George Hill_10/25/2016_@POR', " ['George Hill'", " 'UTA'", ' 2016', ' 10', ' 25', " 'UTA'", " 'POR'", " '38:45'", " '7'", " '18'", " '.389'", " '1'", " '6'", " '.167'", " '4'", " '4'", " '1.000'", " '0'", " '2'", " '2'", " '6'", " '2'", " '1'", " '1'", " '2'", " '19'", " '+2'", ' 0]']
['Rodney Hood_10/25/2016_@POR', " ['Rodney Hood'", " 'UTA'", ' 2016', ' 10', ' 25', " 'UTA'", " 'POR'", " '37:39'", " '9'", " '17'", " '.529'", " '3'", " '7'", " '.429'", " '5'", " '5'", " '1.000'", " '0'", " '2'", " '2'", " '1'", " '2'", " '1'", " '0'", " '3'", " '26'", " '+5'", ' 0]']
['Boris Diaw_10/25/2016_@POR', " ['Boris Diaw'", " 'UTA'", ' 2016', ' 10', ' 25', " 'UTA'", " 'POR'", " '32:10'", " '1'", " '8'", " '.125'"

In [38]:
column_names = 'Player_Name Team Year Month Day Away_Team Home_Team Min_Played FGM FGA FG% 3P 3PA 3P% FT FTA FT% ORB DRB TRB AST STL BLK TOV PT PTS +/- WIN/LOST'.split()

df = pd.read_csv('player_stats_each_game.csv', header=None, names=column_names, index_col=0)
df

,Player_Name,Team,Year,Month,Day,Away_Team,Home_Team,Min_Played,FGM,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PT,PTS,+/-,WIN/LOST
Rudy Gobert_10/25/2016_@POR,['Rudy Gobert','UTA',2016,10,25,'UTA','POR','40:57','4','7',...,'10','14','2','0','2','3','3','12','+7',0]
George Hill_10/25/2016_@POR,['George Hill','UTA',2016,10,25,'UTA','POR','38:45','7','18',...,'2','2','6','2','1','1','2','19','+2',0]
Rodney Hood_10/25/2016_@POR,['Rodney Hood','UTA',2016,10,25,'UTA','POR','37:39','9','17',...,'2','2','1','2','1','0','3','26','+5',0]
Boris Diaw_10/25/2016_@POR,['Boris Diaw','UTA',2016,10,25,'UTA','POR','32:10','1','8',...,'2','3','0','0','0','1','5','2','+1',0]
Joe Johnson_10/25/2016_@POR,['Joe Johnson','UTA',2016,10,25,'UTA','POR','30:30','12','16',...,'4','4','3','1','0','2','0','29','-2',0]
Trey Lyles_10/25/2016_@POR,['Trey Lyles','UTA',2016,10,25,'UTA','POR','17:55','2','6',...,'2','2','2','0','0','3','0','5','-11',0]
Shelvin Mack_10/25/2016_@POR,['Shelvin Mack','UTA',2016,10,25,'UTA','POR','14:48','2','5',...,'1','1','4','2','0','0','1','4','-9',0]
Joe Ingles_10/25/2016_@POR,['Joe Ingles','UTA',2016,10,25,'UTA','POR','14:06','1','1',...,'0','0','1','1','0','1','2','2','-9',0]
Dante Exum_10/25/2016_@POR,['Dante Exum','UTA',2016,10,25,'UTA','POR','8:20','2','4',...,'1','2','0','0','0','0','2','5','-18',0]
Jeff Withey_10/25/2016_@POR,['Jeff Withey','UTA',2016,10,25,'UTA','POR','4:51','0','0',...,'1','1','0','1','1','0','1','0','-11',0]
